<a href="https://colab.research.google.com/github/CHENZhoujing/CNN/blob/main/CNN_Torch_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
batch_size = 64

train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)



In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入1通道，输出10通道，kernel 5*5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        # fully connect
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        # in_size = 64
        in_size = x.size(0) # one batch
        # x: 64*10*12*12
        x = F.relu(self.mp(self.conv1(x)))
        # x: 64*20*4*4
        x = F.relu(self.mp(self.conv2(x)))
        # x: 64*320
        x = x.view(in_size, -1) # flatten the tensor
        # x: 64*10
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
    


def test():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316982
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.493170
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.362335
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.261402
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.187781


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1775, Accuracy: 9488/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.238442
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.164997
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.072519
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.157203
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.193019

Test set: Average loss: 0.1153, Accuracy: 9647/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.143084
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.246259
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.049620
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.197234
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.044428

Test set: Average loss: 0.0865, Accuracy: 9759/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.169888
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.084430
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.060377
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.143334
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.065372

Test set: Average loss: 0.0721, Accuracy: 9771/10000 (98%)

Train Epoch: 5 [0/600